# Tutorial 5

Now, we analysis the diffennt ways we can create or import components.


### Set Up

Lets get some intital imports out the way, and set auto reload, incase we change any of the external utils etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

%load_ext autoreload
%autoreload 2

Now, `pyspice` imports, and setting up a logger.

In [ ]:
import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()

Might need to change the simulator...

In [ ]:
# # # change sim program location depending on system
# if sys.platform == "linux" or sys.platform == "linux2":
#     PySpice.Spice.Simulation.CircuitSimulator.DEFAULT_SIMULATOR = 'ngspice-subprocess'  # needed for linux
# elif sys.platform == "win32":
#     # You will get logging errors/warning, but is should work
#     pass

## Method 0: Locally Defined Components

In [ ]:
# # create the circuit
circuit = Circuit('Tutorial 5.0')

# # add components to the circuit
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

circuit.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)  # Define the 1N4148PH (Signal Diode)
circuit.Diode(1, 1, 2, model='MyDiode')

print(circuit)

In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.dc(Vinput=slice(-3, 3, 0.01)) # where: slice = start:step:stop (inclusive)

plt.plot(np.array(analysis["1"]), np.array(analysis["2"]))
plt.xlabel("Input Voltage (node 1)")
plt.ylabel("Output Voltage (node 2)")

## Method 1: PySpice Library

A library is a directory which is recursively scanned for `.lib` file and
parsed for sub-circuit and models definitions.

A library should be able to be imported using:
```
spice_library = SpiceLibrary('/some/path/')
```
And, to retrieve the file path use: `spice_library['device']` (where name is device.lib)

*Unfortunatly, I cannot seem to get this method to work.
Please refer to the PySpice docs.*

In [ ]:
# from PySpice.Spice.Library import SpiceLibrary


# # # create the circuit
# circuit = Circuit('Tutorial 5.1')

# # # add components to the circuit
# circuit.V('input', 1, circuit.gnd, 10@u_V)
# circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

# spice_library = SpiceLibrary('/lib/')
# circuit.include(spice_library['1n4148'])
# circuit.X('importDiode', '1N4148', 1, 2)

# print(circuit)

## Method 2:  Manually include isung the circuit object

Use `circuit.include()` to add a `.lib` file.

In [ ]:
# # create the circuit
circuit = Circuit('Tutorial 5.2')

# # add components to the circuit
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

pathh = "lib/1N4148.lib"
circuit.include(pathh)  # this adds the system path to the start!!
circuit.X('importDiode', '1N4148', 1, 2)

print(circuit)

In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.dc(Vinput=slice(-3, 3, 0.01)) # where: slice = start:step:stop (inclusive)

plt.plot(np.array(analysis["1"]), np.array(analysis["2"]))
plt.xlabel("Input Voltage (node 1)")
plt.ylabel("Output Voltage (node 2)")

## Method 3: Include using Raw Spice

We can just manually write additional lines to our netlist.

In [ ]:
import os

# # create the circuit
circuit = Circuit('Tutorial 5.3')

# # add components to the circuit
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

new_line = ".include lib/1n4148.lib"
circuit.raw_spice += new_line + os.linesep
circuit.X('importDiode', '1N4148', 1, 2)

print(circuit)

We can use a helpful function to make this slightly simpler:

In [ ]:
from utils.methods import write_line_to_netlist

# # create the circuit
circuit = Circuit('Tutorial 5.3')

# # add components to the circuit
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 2, circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

circuit = write_line_to_netlist(circuit, ".include lib/1n4148.lib")
circuit.X('importDiode', '1N4148', 1, 2)

print(circuit)

In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.dc(Vinput=slice(-3, 3, 0.01)) # where: slice = start:step:stop (inclusive)

plt.plot(np.array(analysis["1"]), np.array(analysis["2"]))
plt.xlabel("Input Voltage (node 1)")
plt.ylabel("Output Voltage (node 2)")